In [1]:
import sys
import warnings
from os import listdir
from os.path import exists

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

warnings.filterwarnings("ignore")

from tqdm import tqdm

In [2]:
def get_weight_parallel(gene, G, corr):
    paths = list(nx.all_simple_paths(G, source="GO:0008150", target=gene))

    if len(paths) != 1:
        path = set()
        for i in paths:
            path = path | set(i)
    else:
        path = paths[0]

    path.remove(gene)
    return [gene] + list(np.average(corr.loc[list(path)], axis=0))

In [3]:
def get_weight(corr_score, go):
    corr_score = pd.read_csv(corr_score, index_col=0)

    nodes = list(set(go[0]) | set(go[1]))
    genes = set(go[["GO" not in i for i in go[1]]][1])
    
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(list(go.itertuples(index=False, name=None)))

    print("Start to calculate the weight of each gene.")
    t = Parallel(n_jobs=-1)(delayed(get_weight_parallel)(gene, G, corr_score) for gene in tqdm(list(genes)))
    t = pd.DataFrame(t)
    t.index = list(t[0])
    t = t.drop(0, axis=1)
    t.columns = corr_score.columns

    weight = pd.concat([corr_score, t])
    weight.to_csv("../DrugCell/data_rcellminer/weight_corr.csv")

In [4]:
def get_graph_info(corr_score, onto_file):
    go = pd.read_table(onto_file, header=None)[[0, 1]]
    go.to_csv("../data/graph.csv",)
    
    get_weight(corr_score, go)

In [5]:
get_graph_info(
    '../DrugCell/data_rcellminer/corr_score.csv', 
    '../DrugCell/data_rcellminer/go.txt'
)

Start to calculate the weight of each gene.


100%|██████████| 2727/2727 [00:46<00:00, 58.85it/s]


In [6]:
pd.read_csv("../DrugCell/data_rcellminer/weight_corr.csv")

,Unnamed: 0,C#CCC(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C(=O)NC(CCC(=O)O)C(=O)O,C(=O)(N)NO,C(C(C(=O)O)N)[N+](=NO)[O-].[Na+],C(C(C(=O)O)NC(=O)CP(=O)(O)O)C(=O)O.[Na+],C(C(C(C(C(CBr)O)O)O)O)Br,C(C1C(C(C(O1)C2=NNC(=C2O)C(=O)N)O)O)O,C(CCCOS(=O)(=O)N)CCCOS(=O)(=O)N,C(CCl)N(C(=O)NC(C=O)C(C(C(CO)O)O)O)N=O,C(CCl)NC(=O)N(CCCl)N=O,...,COC1=C(C=CC(=C1)NS(=O)(=O)C)NC2=C3C=CC=CC3=NC4=CC=CC=C42.CS(=O)(=O)O,COC1=C(C=CC(=C1)NS(=O)(=O)C)NC2=C3C=CC=CC3=NC4=CC=CC=C42.Cl,COC1=C2CN3C(=CC4=C(C3=O)COC(=O)C4(CC(=O)N5CCOCC5)O)C2=NC6=CC=CC=C61,COC1=C2CN3C(=CC4=C(C3=O)COC(=O)C4(CC(=O)OC)O)C2=NC5=CC=CC=C51,COC1=CC(=CC(=C1O)OC)C2C3C(COC3=O)C(C4=CC5=C(C=C24)OCO5)OC6C(C(C7C(O6)COC(O7)C8=CC=CS8)O)O,COC1=CC2=C(C=C1)C3=C(C2=O)C4=C(C=C(C=C4)[N+](=O)[O-])C(=O)N3CCCI,COC1=NC(=NC2=C1N=CN2C3C(C(C(O3)CO)O)O)N,CS(=O)(=O)CS(=O)(=O)OCCCl,CS(=O)(=O)OCCCNCCCOS(=O)(=O)C.Cl,CSC1=CC2=C(C=C1)C3=C(C2=O)C4=C(C=C(C=C4)[N+](=O)[O-])C(=O)N3CCCN=[N+]=[N-]
0,GO:0007005,0.711613,0.649738,0.878147,0.824515,0.970725,0.347863,0.934288,0.775000,0.651379,...,0.917203,0.845584,0.884995,0.611297,0.646067,0.914379,0.796712,0.878745,0.874655,0.881107
1,GO:0006281,0.896026,0.916824,0.936320,0.952285,0.887042,0.231908,0.783535,0.845317,0.859772,...,0.922628,0.899214,0.981981,0.969782,0.856532,0.943300,0.752799,0.923381,0.834372,0.859518
2,GO:0051052,0.897251,0.916824,0.916929,0.973526,0.909714,0.463817,0.834431,0.886754,0.926846,...,0.975529,0.844464,0.957734,0.969782,0.860349,0.986203,0.796712,0.956025,0.914128,0.966376
3,GO:1903047,0.842002,0.953867,0.947401,0.938125,0.945254,0.637748,0.931482,0.895041,0.926846,...,0.912364,0.968835,0.897118,0.969782,0.891822,0.936478,0.796712,0.974679,0.972518,0.909530
4,GO:0006631,0.826532,0.926085,0.936320,0.867070,0.962143,0.231908,0.849185,0.845317,0.920749,...,0.914558,0.800073,0.981981,0.969782,0.883596,0.692523,0.670143,0.830110,0.929703,0.703465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,TXN2,0.851191,0.898700,0.851284,0.850637,0.904904,0.640957,0.854103,0.827381,0.857010,...,0.873396,0.817909,0.885450,0.878796,0.801390,0.900271,0.805538,0.889135,0.883822,0.872461
4809,ID1,0.835386,0.891178,0.871395,0.859445,0.887354,0.561387,0.876235,0.858284,0.885519,...,0.900538,0.835313,0.903158,0.913813,0.805548,0.900500,0.785212,0.912462,0.921898,0.903130
4810,SCN9A,0.878857,0.900805,0.888432,0.891847,0.937794,0.597444,0.845880,0.891091,0.915239,...,0.917670,0.857487,0.855459,0.887786,0.822560,0.920378,0.832898,0.901079,0.905935,0.882550
4811,DAPK3,0.879348,0.918076,0.884203,0.871841,0.910119,0.566259,0.894085,0.881526,0.890461,...,0.915620,0.866458,0.902819,0.900218,0.844131,0.925382,0.818186,0.912973,0.918983,0.912792
